In [1]:
# !jupyter nbextension enable --py widgetsnbextension --sys-prefix
# !jupyter serverextension enable voila --sys-prefix

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import base64

In [4]:
import cv2

In [5]:
import ipywidgets as widgets
from ipywidgets import Layout, Button, Box, Text, HBox, VBox, Image

In [6]:
from IPython.display import display, clear_output
from IPython.display import HTML

In [7]:
from lib.image import image_data
from lib.analysis import dyssynchrony_analysis

In [8]:
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 20)

In [9]:
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
warnings.filterwarnings("ignore", category=RuntimeWarning) 

In [40]:
def load_cv2_image_from_bytes(bytes_):
    return cv2.imdecode(np.frombuffer(bytes_, dtype=np.uint8), cv2.IMREAD_COLOR)

def get_uploader_content(uploader):
    return uploader.value[list(uploader.value)[0]]["content"]

def display_html(html):
    display(widgets.HTML(value=html))
    
def create_download_link( df, title = "Download CSV file", filename = "data.csv"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)
    
def create_inttext(description, min = None, max = None):
    return widgets.BoundedIntText(
        value=0,
        min = min,
        max = max,
        description=description,
        disabled=False
    )
    
def plot_analysis_result_slice(df,slice_number):
    
    fig, ((ax1, ax2),(ax3, ax4)) = plt.subplots(2,2, figsize=(20,10))

    ax1.plot(df.loc[slice_number, "amplitudes"])
    ax1.set_title('amplitudes')
    ax1.grid()

    # log y axi
    ax2.plot(df.loc[slice_number, "intensities"])
    ax2.set_title('Intensities')
    ax2.grid()

    ax3.plot(df.loc[slice_number, "max_peaks_positions"])
    ax3.set_title('max_peaks_positions')
    ax3.grid()

    ax4.plot(df.loc[slice_number, "min_peaks_positions"])
    ax4.set_title('max_peaks_positions')
    ax4.grid()

    plt.show()    

def read_progress_img():
    animatedGif = "./photos_examples/01-progress.gif"
    with open(animatedGif , "rb") as file:
        return file.read()

def create_image(image):
    return Image(
        value=image,
        format='gif',
        width=100,
        height=100)

    
def display_results(analysis_results):
    """
    Displays the results into the results_box. 
    Every print or display will be automatically redirected to it. 
    """
    results_box.clear_output()
    
    # this is just a sample table...
    display_html("<h2>Slices Table</h2>")
    slices = analysis_results["slices"] 
    display(slices)
    
    with_progress(lambda: display(create_download_link(last_analysis_results["slices"])))    
    
    display_html("<h2 style='margin-top:20px; margin-botton:15px'>Slices plot</h2>")
    slice_selector = create_inttext("Plot slice: ", min = 0, max = len(slices) - 1)
    plot_btn = widgets.Button(description="Plot", button_style='danger')
    plot_box = widgets.Output()

    
    @plot_box.capture()
    def on_button_clicked(_button):
        plot_box.clear_output()

        display_html("<h2>Slice plot</h2>")
        with_progress(lambda: plot_analysis_result_slice(slices, slice_selector.value))
        
        
    plot_btn.on_click(on_button_clicked)
    
    display(slice_selector)
    display(plot_btn)
    display(plot_box)

def with_progress(code):
    progress = create_image(read_progress_img())
    display(progress)
    code()
    progress.close()
    
def clear_uploader(uploader):
    uploader.value.clear()
    uploader._counter = 0

def cardiaAp_analyzer(image):
    return dyssynchrony_analysis.analyze_image(image, \
                                               min_dist_between_maxs.value, \
                                               calibration.value, \
                                               slice_width=slice_width.value)

In [41]:
uploader = widgets.FileUpload(accept='.tif', multiple=False)

slice_width = widgets.IntText(
    value=1,
    description='Slice width:',
    disabled=False
)

min_dist_between_maxs = widgets.IntText(
    value=0,
    description='Dist between maxs:',
    disabled=False
)

calibration = widgets.IntText(
    value=0,
    description='Calibration:',
    disabled=False
)

In [56]:
html_text_upload = "<p style='font-size:15px; color:black'> Upload your image to initialize the analysis</p>"
header_text_upload = widgets.HTML(value=html_text_upload)

box_upload = Layout(display='flex',
                    flex_flow='flex-wrap',
                    width='50%',
                    margin='0px 10px 10px 0px',
                    padding=' 2% 5% 0 5%',
                    justify_content='space-between')

vbox_upload = widgets.VBox([header_text_upload, uploader], layout=box_upload)

In [57]:
box_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    width='50%',
                    margin='0px 10px 10px 0px',
                    padding=' 2% 5% 0 5%',
                    justify_content='space-between')

vbox_settings = widgets.VBox([slice_width, min_dist_between_maxs, calibration], layout=box_layout)

In [60]:
analyze_btn = widgets.Button(description="Analyze", button_style='danger', layout=Layout(padding = '2% 5% 0 5%'))
results_box = widgets.Output()

@results_box.capture()
def on_button_clicked(_button):
    """
    Runs analysis and displays results. 
    It stores them in last_analysis_results
    for debugging purposes
    """
    global last_analysis_results
    results_box.clear_output()

    if uploader._counter == 0:
        print("Please upload an image first")
    else:
        print("Anayzing, please wait...")
        raw_results = cardiaAp_analyzer(load_cv2_image_from_bytes(get_uploader_content(uploader)))
        last_analysis_results = {
            "raw": raw_results,
            "slices":  pd.DataFrame(raw_results["slices"])
        }
        display_results(last_analysis_results)
        clear_uploader(uploader)
        
analyze_btn.on_click(on_button_clicked)

In [61]:
clear_uploader(uploader)
display_html("<h1 style='font-size:25px; color:black; padding: 2% 5% 0 5%'> Welcome to Cardi<b>AP</b> </h1>")

display(vbox_upload)
display(vbox_settings)

display(analyze_btn)
display(results_box)

HTML(value="<h1 style='font-size:25px; color:black; padding: 2% 5% 0 5%'> Welcome to Cardi<b>AP</b> </h1>")

Button(button_style='danger', description='Analyze', layout=Layout(padding='2% 5% 0 5%'), style=ButtonStyle())

Output()